In [ ]:
!pip install pytorch_metric_learning -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.3/119.3 kB 1.1 MB/s eta 0:00:00


### Импорт данных

In [ ]:
import numpy as np
data = np.load('/content/drive/MyDrive/hackaton_mrt/ihb.npy')

# В каких семплах есть np.nan?
nan_rows = np.any(np.isnan(data), axis=1)
has_na_indexes = np.unique(np.where(nan_rows)[0])
is_nan_labels = np.array([1 if i in has_na_indexes else 0 for i in range(data.shape[0])])

### Выравнивание фичей по парам векторов

Идея следующая: с помощью T-SNE **наблюдения легко мэтчатся парами**.

Среди этих пар есть **совпадения между семплами, сделанными по разным атласам** (brainstomme и по Shaefer200).

Так, можно совершить отображение из пространства $\mathbb{R}^{246} \to \mathbb{R}^{200}$ с минимальной потерей информации.

Делать это мы будем ориентируясь лишь на корреляции во "временном" измерени.

In [ ]:
# Вычисляем корреляционные матрицы для каждого семпла
correlations = []

for sample in data:
    sample_data = np.nan_to_num(sample, 0)
    corr_matrix = np.cov(sample_data)
    correlations.append(corr_matrix.flatten())

correlations = np.array(correlations)
correlations.shape

(320, 100)

In [ ]:
# Применяем к данным T-SNE
from sklearn.manifold import TSNE

# Применяем уменьшение размерности до 3D
dim_reducer = TSNE(
    n_components = 2,
    random_state = 42,
    perplexity = 1,
    max_iter = 2000
)

X_reduced = dim_reducer.fit_transform(correlations)

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import plotly.express as px
from tqdm.auto import tqdm

# Кластеризация чтобы сматчить пары
clusterer = AgglomerativeClustering(
    linkage='ward',
    n_clusters=None,
    distance_threshold=7
)
labels_clusters_align = clusterer.fit_predict(X_reduced)

# Визуализация, что пары мэтчат векторы 200 и 246
px.scatter(
    X_reduced,
    x=0, y=1,
    color=is_nan_labels,
    width=500,
    height=500
).show()

# Визуализация кластеров- мэтчей
px.scatter(
    X_reduced,
    x=0, y=1,
    color=labels_clusters_align,
    width=500,
    height=500
).show()

In [ ]:
# @title Отбираем по одному вектору из каждой группы (будем учиться различать *группы*)
labels_to_find = set(labels_clusters_align)
found_args = []

for arg, cluster_label in enumerate(labels_clusters_align):
    if cluster_label in labels_to_find:
        labels_to_find.remove(cluster_label)
        found_args.append(arg)

found_args = np.array(found_args)

In [ ]:
# Отбираем только кластеры, которые содержат несколько меток
def check_is_both_here(label, is_nan_labels):
    """ Проверяет, есть ли в кластере одновременно семплы с NaN и без NaN. """
    return all([
        0 in is_nan_labels[labels_clusters_align == label],
        1 in is_nan_labels[labels_clusters_align == label]
    ])

vectors = []
vectors_labels = []

for label in labels_clusters_align:
    if not check_is_both_here(label, is_nan_labels):
        continue

    vectors.append(data[labels_clusters_align == label])
    vectors_labels.append(is_nan_labels[labels_clusters_align == label])

In [ ]:
def collect_vector_pairs(vectors, labels):
    """
    Собирает пары векторов с метками 0 и 1 из групп векторов и соответствующих меток.

    Параметры:
    vectors (list of np.ndarray): Список массивов векторов для каждой группы.
    labels (list of np.ndarray): Список массивов меток (0 или 1) для каждой группы.

    Возвращает:
    X_0 (np.ndarray): Векторы с меткой 0, собранные из всех групп.
    X_1 (np.ndarray): Векторы с меткой 1, соответствующие в каждой группе.
    """

    X_0_list = []
    X_1_list = []

    for group_vectors, group_labels in zip(vectors, labels):
        # Получаем векторы с меткой 0 и 1 для текущей группы
        vectors_0 = group_vectors[group_labels == 0]
        vectors_1 = group_vectors[group_labels == 1]

        # Берем минимальное количество векторов между метками 0 и 1, чтобы составить пары
        min_len = min(len(vectors_0), len(vectors_1))

        # Собираем пары
        X_0_list.append(vectors_0[:min_len])
        X_1_list.append(vectors_1[:min_len])

    # Объединяем все группы в один массив
    X_0 = np.vstack(X_0_list)
    X_1 = np.vstack(X_1_list)

    return X_0, X_1

# Собираем пары векторов
X_0, X_1 = collect_vector_pairs(vectors, vectors_labels)

In [ ]:
# @title Обучаем отображение $\mathbb{R}^{246} \to \mathbb{R}^{200}$
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# Данные для обучения (пары векторов)
X = X_0
Y = X_1[:, :, :200]

# Параметры данных
i, j, m = X.shape
_, _, k = Y.shape

# Разворачиваем X и Y в двумерные массивы для обучения модели
X_reshaped = X.reshape(i * j, m)
Y_reshaped = Y.reshape(i * j, k)

# Создаем модель нейросети
model = Sequential()
model.add(Input(shape=(m,)))
model.add(Dense(k, activation='leaky_relu'))
model.add(Dense(k, activation='leaky_relu'))
model.add(Dense(k, activation='leaky_relu'))

# Компилируем модель
model.compile(optimizer=Adam(), loss='mean_squared_error')
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                     │ (None, 200)                 │          49,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 200)                 │          40,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 200)                 │          40,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 129,800 (507.03 KB)

 Trainable params: 129,800 (507.03 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    restore_best_weights = True
)

history = model.fit(
    X_reshaped,
    Y_reshaped,
    epochs=50,
    batch_size=64,
    verbose=1,
    validation_split=0.1,
    callbacks=[early_stopping]
)

Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.9914 - val_loss: 0.7442
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.6956 - val_loss: 0.5936
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5723 - val_loss: 0.5196
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4954 - val_loss: 0.4715
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4480 - val_loss: 0.4361
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4134 - val_loss: 0.4088
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3943 - val_loss: 0.3866
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3661 - val_loss: 0.3677
Epoch 9/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3480 - val_loss: 0.3522
Epoch 10/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3335 - val_loss: 0.3392
Epoch 11/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3171 - val_loss: 0.3261
Epoch 12/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3035 - val

In [ ]:
import plotly.express as px

px.line(
    history.history,
    y=['loss', 'val_loss'],
    width=800,
    height=600,
    range_y=[0, max([max(x) for x in history.history.values()])]
)

**Модель следовало дообучить до EarlyStopping!!!**

In [ ]:
# Ремаппим все векторы в пространство размерности 200
data_nnan = data[is_nan_labels == 0]

i_nnan, j_nnan, m_nnan = data_nnan.shape
data_nnan_reshaped = data_nnan.reshape(i_nnan * j_nnan, m_nnan)
predictions_nnan = model.predict(data_nnan_reshaped)
remapped_vectors_nnan = predictions_nnan.reshape(i_nnan, j_nnan, predictions_nnan.shape[-1])

# Для векторов с NaN используем единичную матрицу
remapped_vectors_nan = np.einsum("ijm,mk->ijk", data[is_nan_labels == 1], np.eye(m))

# Собираем итоговые данные
data_remapped = data[:, :, :200]
data_remapped[is_nan_labels == 0] = remapped_vectors_nnan
data_remapped[is_nan_labels == 1] = remapped_vectors_nan[:, :, :200]

50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [ ]:
# @title Предыдущий вариант — линейная модель (МНК)
# import numpy as np

# def solve_for_A(X, Y):
#     """
#     Находит матрицу A в уравнении Y = X A с помощью метода наименьших квадратов (SVD).

#     Параметры:
#     X (np.ndarray): Массив X (размерность (i, j, m)).
#     Y (np.ndarray): Массив Y (размерность (i, j, k)).

#     Возвращает:
#     A (np.ndarray): Матрица A, найденная методом наименьших квадратов.
#     """
#     # Преобразуем X и Y в двумерные массивы
#     i, j, m = X.shape
#     _, _, k = Y.shape

#     # Развернем X в двумерный массив размерности (i*j, m)
#     X_reshaped = X.reshape(i * j, m)

#     # Развернем Y в двумерный массив размерности (i*j, k)
#     Y_reshaped = Y.reshape(i * j, k)

#     # Находим псевдообратную матрицу X с помощью SVD
#     X_pseudo_inv = np.linalg.pinv(X_reshaped)

#     # Вычисляем A
#     A = np.dot(X_pseudo_inv, Y_reshaped)

#     return A

# X = X_0
# Y = X_1[:, :, :200]

# # Решаем уравнение на наименьшие квадраты
# A = solve_for_A(X, Y)
# I = np.eye(A.shape[0])

# # Преобразуем векторы 246 в векторы 200
# # Единичная матрица нужна, чтобы быть точно уверенным, что не накосячили с порядком индексов
# remapped_vectors_nnan = np.einsum("ijm,mk->ijk", data[is_nan_labels == 0], A)
# remapped_vectors_nan = np.einsum("ijm,mk->ijk", data[is_nan_labels == 1], I)

# # Вставляем пере
# data_remapped = data[:, :, :200]
# data_remapped[is_nan_labels == 0] = remapped_vectors_nnan
# data_remapped[is_nan_labels == 1] = remapped_vectors_nan[:, :, :200]

In [ ]:
# # Разделение данных на тренировку и валидацию
# data_remapped_doubled = np.concatenate([np.empty_like(data_remapped[:, :5, :])] * 2)
# data_remapped_doubled[0::2] = data_remapped[:, :5, :]
# data_remapped_doubled[1::2] = data_remapped[:, 5:, :]

# # Удвоение меток
# doubled_labels = np.array([i//2 for i in range(len(data_remapped_doubled))])
# doubled_is_nan_labels = np.array([is_nan_labels[i//2] for i in range(len(data_remapped_doubled))])

### Векторизация временных рядов

Используем подход из [статьи](https://onlinelibrary.wiley.com/doi/epdf/10.1002/hbm.26561):

- Считаем матрицу корреляций в "пространственном" измерении ($246\times 246$)
- Вытягиваем верхнетреугольную часть в вектор размерности $\frac{(246)(246-1)}{2}$
- Берём $\text{arctanh}$ от всех коэффициентов корреляции (дабы особо выделить близкие к $1$ значения)
- Совершаем $z$-преобразование Фишера (среднее в ноль, дисперсия в единицу)

In [ ]:
import numpy as np
from scipy.stats import zscore
import random

# Вычисляем корреляционные матрицы для каждого семпла
def differ(array, n = 1, prepend = np._NoValue):
    if n == 0:
        return array
    elif n == 1:
        return np.diff(array, axis=1, prepend=prepend)

    result = array
    for _ in range(n):
        result = differ(result, n=1, prepend=prepend)
    return result

def calculate_fc(corr_matrix):
    fc_vector = np.arctanh(corr_matrix)
    return zscore(fc_vector)

def vectorize_sample(sample, diff_n = 0):
    sample_data = differ(sample, n=diff_n)
    corr_matrix = np.corrcoef(sample_data.T)
    upper_triangle_indices = np.triu_indices_from(corr_matrix, k=1)
    vectorized = corr_matrix[upper_triangle_indices]
    ficher_transform = calculate_fc(vectorized)
    return np.array(ficher_transform)

window_size = random.choice(range(5, 10))
start_point = random.randint(0, 10 - window_size)

vectorize_sample(data_remapped[0][start_point : start_point + window_size, :])

array([ 1.700714  , -0.12323163,  2.95762535, ...,  1.15378141,
        1.96450736,  0.51367903])

------

### Нейросеть для Fingerprint'инга

Далее **обучаем нейросеть**.

Она строится по принципу **metric learning**: векторизуем корреляции по семплу, прогоняем через модель, получаем Embedding семпла.

Пытаемся обучить модель так, чтобы она отображала **как можно ближе** векторы, соответствующие **одному** семплу и **как можно дальше** векторы, принадлежащие **разным**.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_metric_learning import losses

# Определение архитектуры сети
class FCFingerprintNet(nn.Module):
    def __init__(
            self,
            input_size=62835,
            hidden_size=2000,
            output_size=300,
            dropout_rate = 0.1
        ):
        super(FCFingerprintNet, self).__init__()
        # Входной слой
        self.fc1 = nn.Linear(input_size, hidden_size)
        # Первый скрытый слой
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        # Выходной слой
        self.fc3 = nn.Linear(hidden_size, output_size)
        # Функция активации
        self.activation = nn.Tanh()
        # Dropout слой
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.dropout(x)
        x = self.activation(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
import torch
import random
from torch.utils.data import Dataset

class RandomWindowDataset(Dataset):
    def __init__(self, data, N, window_min=5, window_max=10):
        """
        Args:
            data (torch.Tensor or np.ndarray): Входные данные (например, тензор или массив).
            vectorize_fn (function): Функция для векторизации выборки.
            N (int): Количество случайных окон для каждого примера.
            window_min (int): Минимальный размер окна.
            window_max (int): Максимальный размер окна.
        """
        self.data = data
        self.N = N
        self.window_min = window_min
        self.window_max = window_max

    def __len__(self):
        return len(self.data)

    @staticmethod
    def vectorize_sample(sample, diff_n = 0):
        sample_data = differ(sample, n=diff_n)
        corr_matrix = np.corrcoef(sample_data.T)
        upper_triangle_indices = np.triu_indices_from(corr_matrix, k=1)
        vectorized = corr_matrix[upper_triangle_indices]
        vectorized = np.arctanh(vectorized)
        return np.array(vectorized)

    def __getitem__(self, idx):
        # Получаем данные по индексу
        sample = self.data[idx]
        vectorized_windows = []

        # Выбираем N случайных окон
        for _ in range(self.N):
            window_size = random.choice([7, 8, 9, 10])
            start_point = random.randint(0, sample.shape[0] - window_size)
            windowed_sample = sample[start_point : start_point + window_size, :]

            # Применяем функцию векторизации
            vectorized_sample = self.vectorize_sample(windowed_sample)
            vectorized_windows.append(vectorized_sample)

        vectorized_windows = np.array(vectorized_windows)

        # Возвращаем список N векторизованных окон
        return torch.tensor(vectorized_windows)

Поскольку матрица корреляций должна совпадать для разных временных окон, будем считать её для разных окон для одних и тех же семплов.

Так мы получим естественную разметку и сможем обучить низкоразмерные Embedding'и.


**ДРУГИЕ АУГМЕНТАЦИИ И ТЕХНИКИ:**

- **НЕ отбирать по ОДНОМУ вектору из каждой группы**!!! Рассматривать группы целиком!

- Миксить векторы **уже спаренных** семплов ($\beta$-взвешенные средние векторизованных матриц корреляций)
- Миксить данные **уже спаренных** семплов во временной области ($\beta$-взвешенные средние редуцированных в $\mathbb{R}^{200}$ наблюдений)
- Зашумлять данные **гауссовым шумом** (во временной, в частотной областях)
- **Псевдолейблинг** данных с помощью простой модели

In [ ]:
def custom_collate_fn(batch):
    """
    Функция объединяет N случайных окон и возвращает:
    1. Объединенный тензор из всех окон.
    2. Тензор с метками, указывающий на исходные семплы для каждого окна.
    """
    all_windows = []
    all_labels = []

    for idx, windows in enumerate(batch):
        for window in windows:
            all_windows.append(window)
            all_labels.append(idx)  # Добавляем метку для каждого окна (индекс исходного семпла)

    # Объединяем все окна в один тензор
    all_windows_tensor = torch.stack(all_windows).to(torch.float32)

    # Превращаем список меток в тензор
    all_labels_tensor = torch.tensor(all_labels, dtype=torch.long)

    return all_windows_tensor, all_labels_tensor

In [ ]:
import torch
from torch.utils.data import DataLoader

dataset = RandomWindowDataset(data_remapped[found_args], N=3)
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=8, shuffle=True, collate_fn=custom_collate_fn)

In [ ]:
# Установка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создание модели
model = FCFingerprintNet(
    input_size = dataset[1].shape[-1],
    hidden_size = 2600,
    output_size = 16,
    dropout_rate = 0.3
).to(device)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
loss_func = losses.TripletMarginLoss(margin=1, smooth_loss=True)  # Используем TripletLoss

In [ ]:
from tqdm.auto import tqdm
from time import sleep

model.train()

for epoch in range(3):
    running_loss = 0.0

    # Используем tqdm для отслеживания прогресса цикла
    progress_bar = tqdm(enumerate(data_loader), desc=f"Epoch {epoch+1}")

    for i, (vectors, labels) in progress_bar:
        vectors, labels = vectors.to(device), labels.to(device)
        optimizer.zero_grad()

        embeddings = model(vectors)
        if embeddings.isnan().any():
            continue

        # Вычисление потерь
        loss = loss_func(embeddings, labels)

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        # Накопление потерь для вывода среднего значения
        running_loss += loss.item()

        # Обновляем описание прогресс-бара с текущими потерями
        progress_bar.set_postfix({'loss': loss.item(), 'run_loss': running_loss / (i + 1)})

Epoch 1: 0it [00:00, ?it/s]

Epoch 2: 0it [00:00, ?it/s]

Epoch 3: 0it [00:00, ?it/s]

In [ ]:
with torch.no_grad():
    model.eval()
    vectorized_samples = torch.tensor(np.array(
        [dataset.vectorize_sample(x) for x in data_remapped]), dtype=torch.float32)
    embeddings = model(vectorized_samples)

### Комитеты KMeans

Далее есть много способов кластеризовать Embedding'и.

Рассмотрим два. Первый — **KMeans**.

Поскольку он стохастический, будем **ансамблировать предсказания** многих одинаковых моделей (**Majority Voting**).

In [ ]:
# Применяем к данным понижение размерности
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
import plotly.express as px
from tqdm.auto import tqdm

labels_to_align = []

# Применяем уменьшение размерности до 3D
for random_state in tqdm(range(100)):
    clusterer = KMeans(n_clusters=40, random_state=random_state)
    labels_to_align.append(clusterer.fit_predict(embeddings))


labels_aligned = [labels_to_align[0]]

for labels_ in labels_to_align[1:]:

    # Выравниваем метки через confusion matrix
    conf_matrix = confusion_matrix(
        labels_to_align[0],
        labels_
    )
    # Добавляем выравненные метки в список
    labels_aligned.append(
        conf_matrix.argmax(axis=0)[labels_]
    )

# Голосуем комитетами за итоговую метку
real_labels = np.array(
    [np.bincount(arr).argmax() for arr in np.array(labels_aligned).T])

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# Рисуем матрицу попарных расстояний
cosine_distances_ = cosine_distances(embeddings[real_labels.argsort()])
px.imshow(cosine_distances_)

Второй — **агломеративная (иерархическая) кластеризация**.

Она удобна тем, что позволяет выбрать threshold, по которому будет объединять объекты выборки в кластеры.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

distances = cosine_distances(embeddings)

clusterer = AgglomerativeClustering(
    n_clusters=None,
    distance_threshold=0.3,
    metric='precomputed',
    linkage='complete'
)

labels_pred = clusterer.fit_predict(
    np.where(distances < 0.5, distances, 2)
)

cosine_distances_ = euclidean_distances(embeddings[labels_pred.argsort()])
px.imshow(cosine_distances_)

In [ ]:
# Сохраняем предсказания в CSV
import pandas as pd

filename = f'submit.csv'
pd.DataFrame({'prediction': labels_pred}).to_csv(filename, index=False)
print(f'Сохранены предсказания в файл {filename}')

Сохранены предсказания в файл submit.csv
